In [1]:
import torch.nn as nn
import torch
import torch.nn.functional as F
import numpy as np
import pandas as pd
import nltk
import matplotlib.pyplot as plt

from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from torch.utils.data import TensorDataset, DataLoader, Subset
from tensorflow.keras.utils import pad_sequences

from torch.utils.data import TensorDataset, DataLoader, Subset
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


tqdm.pandas()

# 데이터 Load

In [2]:
from datasets import load_dataset

dataset = load_dataset("bentrevett/multi30k")

In [3]:
from transformers import AutoModel, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-de")

d:\anaconda\envs\Pray\Lib\site-packages\transformers\models\marian\tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [4]:
dataset2 = dataset.map(lambda e: tokenizer(e['en'], padding= False), batched=True)
dataset3 = dataset.map(lambda e: tokenizer(e['de'], padding= False), batched=True)

In [5]:
58101 # 이를 start token으로 지정.

58101

In [6]:
# np.unique([len(i) for i in dataset2['train']['input_ids']]), np.unique([len(i) for i in dataset3['train']['input_ids']])

In [7]:
def pad_sequences(sequences):
    max_len = 128
    padded_sequences = []
    for sequence in sequences:
        if len(sequence) >= max_len:
            padded_sequence = [58101] + sequence[:max_len-1]  # 최대 길이까지 잘라냄
        else:
            padded_sequence = [58101] + sequence + [58100] * (max_len - len(sequence) - 1)  # 패딩 추가
        padded_sequences.append(padded_sequence)
    return padded_sequences

In [8]:
padded_batch = pad_sequences(dataset2['train']['input_ids'])

In [9]:
train_en, train_ge = torch.tensor(pad_sequences(dataset2['train']['input_ids'])), torch.tensor(pad_sequences(dataset3['train']['input_ids']))
valid_en, valid_ge = torch.tensor(pad_sequences(dataset2['validation']['input_ids'])), torch.tensor(pad_sequences(dataset3['validation']['input_ids']))
test_en, test_ge = torch.tensor(pad_sequences(dataset2['test']['input_ids'])), torch.tensor(pad_sequences(dataset3['test']['input_ids']))

In [10]:
set([len(i) for i in train_en]), set([len(i) for i in train_ge])

({128}, {128})

In [11]:
class Dataset(Dataset):
    
    def __init__(self, inputs, output):

        self.inputs = inputs
        self.output = output

    def __getitem__(self, idx):

        inputs = self.inputs[idx]
        output = self.output[idx]
        
        return inputs, output

    def __len__(self):
        return len(self.output)

In [12]:
train_dataset = Dataset(train_en, train_ge)
valid_dataset = Dataset(valid_en, valid_ge)
test_dataset = Dataset(test_en, test_ge)

In [13]:
train_loader = DataLoader(train_dataset, batch_size = 64, shuffle = True, drop_last = True)
valid_loader = DataLoader(valid_dataset, batch_size = 64, shuffle = True, drop_last = True)
test_loader = DataLoader(test_dataset, batch_size = 64, shuffle = False, drop_last = True)

# Model

In [14]:
import math

class Embeddings(nn.Module):


    def __init__(self, vocab_size, d_model, max_len = 128):
        super(Embeddings, self).__init__()
        self.d_model = d_model
        self.dropout = nn.Dropout(0.1)
        self.embed = nn.Embedding(vocab_size + 1, d_model)
        self.pe = self.create_positinal_encoding(max_len, self.d_model)
        self.dropout = nn.Dropout(0.1)

    def create_positinal_encoding(self, max_len, d_model):
        pe = torch.zeros(max_len, d_model).to(device)
        for pos in range(max_len):  
            for i in range(0, d_model, 2):  
                pe[pos, i] = math.sin(pos / (10000 ** ((2 * i)/d_model)))
                pe[pos, i + 1] = math.cos(pos / (10000 ** ((2 * (i + 1))/d_model)))
        pe = pe.unsqueeze(0)  
        return pe

    def forward(self, encoded_words):
        
        embedding = self.embed(encoded_words) * torch.sqrt(torch.tensor(self.d_model)).to(device)
        embedding += self.pe[:, :embedding.size(1)]   
        embedding = self.dropout(embedding)
        return embedding

In [15]:
# class Attention(nn.Module):
    
#     def __init__(self, embedding_size = 512):
        
#         self.data = data
#         self.embedding_size= embedding_size
#         self.weight_Q = nn.Linear(embedding_size, embedding_size)
#         self.weight_K = nn.Linear(embedding_size, embedding_size)
#         self.weight_V = nn.Linear(embedding_size, embedding_size)
#         self.softmax = nn.Softmax()
        
#     def forward(self, data):
        
#         Q = self.weight_Q(data)
#         K = self.weight_K(data)
#         V = self.weight_V(data)
#         score = torch.matmul(Q,K.T) / torch.sqrt(self.embedding_size)
#         value = self.softmax(score) * V
#         return value
         

In [16]:
def create_masks(inputs, outputs_input, outputs_target):
    
    def subsequent_mask(size):
        mask = torch.triu(torch.ones(size, size)).transpose(0, 1).type(dtype = torch.uint8)
        return mask.unsqueeze(0) # 상삼각행렬 생성 -> 행과 열을 뒤 바꾸어 하삼각행렬로 바꿈. (밑에가 다 0)
    
    inputs_mask = inputs != 58100
    inputs_mask = inputs_mask.to(device)
    inputs_mask = inputs_mask.unsqueeze(1).unsqueeze(1) # 각  input에 대해서 상삼각행렬에 대응하도록 설정.
    
    outputs_input_mask = outputs_input != 58100
    outputs_input_mask = outputs_input_mask.unsqueeze(1) 
    outputs_input_mask = outputs_input_mask & subsequent_mask(outputs_input.size(-1)).type_as(outputs_input_mask.data)
    outputs_input_mask = outputs_input_mask.unsqueeze(1)
    # masking을 해줌으로서, 

    
    outputs_target_mask = outputs_target != 58100
    
    return inputs_mask, outputs_input_mask, outputs_target_mask
        
        

In [17]:
class MultiHeadAttention(nn.Module):
    
    def __init__(self, heads, d_model):
        super(MultiHeadAttention, self).__init__()
        self.d_k = d_model // heads
        self.heads = heads
        self.dropout = nn.Dropout(0.1)
        self.query = nn.Linear(d_model, d_model)
        self.key  = nn.Linear(d_model, d_model)
        self.value = nn.Linear(d_model, d_model)
        self.concat = nn.Linear(d_model, d_model)
        
    def forward(self, query, key, value, mask):
        
        query = self.query(query)
        key = self.key(key)
        value = self.value(value)
        
        query = query.view(query.shape[0], -1, self.heads, self.d_k).permute(0, 2, 1, 3)
        key = key.view(key.shape[0], -1, self.heads, self.d_k).permute(0, 2, 1, 3)
        value = value.view(value.shape[0], -1, self.heads, self.d_k).permute(0, 2, 1, 3)
        

        scores = torch.matmul(query, key.permute(0 ,1 ,3, 2)) / math.sqrt(query.size(-1))
        
        # print(query.shape, key.shape, value.shape, mask.shape, scores.shape)

        
        scores = scores.masked_fill(mask == 0, -1e9) # masking 된 것에 매우 작은 수 부여 -> softmax 계산시 -inf 로 계산되어짐.
        weights = F.softmax(scores, dim = -1) # attention score 계산
        context = torch.matmul(weights, value)  # attention value 계산
        context = context.permute(0,2,1,3).contiguous().view(context.shape[0], -1, self.heads * self.d_k)
        
        interacted = self.concat(context)
        
        return interacted
        
        

In [18]:
class FeedForward(nn.Module):
    
    def __init__(self, d_model, middle_dim = 2048):
        super(FeedForward, self).__init__()
        
        self.fc1 = nn.Linear(d_model, middle_dim)
        self.fc2 = nn.Linear(middle_dim, d_model)
        self.dropout = nn.Dropout(0.1)
        
    def forward(self,x):
        out = F.relu(self.fc1(x))
        out = self.fc2(self.dropout(out))
        return out        

In [19]:
class EncoderLayer(nn.Module):
    
    def __init__(self, d_model, heads):
        super(EncoderLayer, self).__init__()
        self.layernorm = nn.LayerNorm(d_model)
        self.self_multihead = MultiHeadAttention(heads, d_model)
        self.feed_forward = FeedForward(d_model)
        self.dropout = nn.Dropout(0.1)
        
    def forward(self, embeddings, mask):
        interacted = self.dropout(self.self_multihead(embeddings, embeddings, embeddings, mask))
        interacted = self.layernorm(interacted + embeddings)
        feed_forward_out = self.dropout(self.feed_forward(interacted))
        encoded = self.layernorm(feed_forward_out + interacted)
        return encoded
        

In [20]:
class DecoderLayer(nn.Module):
    
    def __init__(self, d_model, heads):
        super(DecoderLayer, self).__init__()
        self.layernorm = nn.LayerNorm(d_model)
        self.self_multihead = MultiHeadAttention(heads, d_model)
        self.src_multihead = MultiHeadAttention(heads, d_model)
        self.feed_forward = FeedForward(d_model)
        self.dropout = nn.Dropout(0.1)
        
    def forward(self, embeddings, encoded, src_mask, target_mask):
        query = self.dropout(self.self_multihead(embeddings, embeddings, embeddings, target_mask))
        query = self.layernorm(query + embeddings)
        interacted = self.dropout(self.src_multihead(query, encoded, encoded, src_mask))
        interacted = self.layernorm(interacted + query)
        feed_forward_out = self.dropout(self.feed_forward(interacted))
        decoded = self.layernorm(feed_forward_out + interacted)
        return decoded

In [21]:
class P_Transformer(nn.Module):
    
    def __init__(self, d_model, heads, num_layers, vocab_size):
        super(Transformer, self).__init__()
        
        self.d_model = d_model
        self.vocab_size = vocab_size + 1
        self.embed = Embeddings(self.vocab_size, d_model)
        self.encoder = nn.ModuleList([EncoderLayer(d_model, heads) for _ in range(num_layers)])
        self.decoder = nn.ModuleList([DecoderLayer(d_model, heads) for _ in range(num_layers)])
        self.logit = nn.Linear(d_model, self.vocab_size)
        
    def encode(self, src_words, src_mask):
        src_embeddings = self.embed(src_words)
        encoded_layers = []
        for layer in self.encoder:
            src_embeddings = layer(src_embeddings, src_mask)
            encoded_layers.append(src_embeddings)
            
        return encoded_layers

    def decode(self, target_words, target_mask, src_embeddings, src_mask):
        tgt_embeddings = self.embed(target_words)
        for i, layer in enumerate(self.decoder):
            tgt_embeddings = layer(tgt_embeddings, src_embeddings[i], src_mask, target_mask)
            decoded_layers.append(tgt_embeddings)
        return tgt_embeddings

    def forward(self, src_words, src_mask, target_words, target_mask):
        encoded_layers = self.encode(src_words, src_mask)
        decoded = self.decode(target_words, target_mask, encoded_layers, src_mask)
        out = F.log_softmax(self.logit(decoded), dim = 2)
        return out

In [22]:
class Transformer(nn.Module):
    
    def __init__(self, d_model, heads, num_layers, vocab_size):
        super(Transformer, self).__init__()
        
        self.d_model = d_model
        self.vocab_size = vocab_size + 1
        self.embed = Embeddings(self.vocab_size, d_model)
        self.encoder = nn.ModuleList([EncoderLayer(d_model, heads) for _ in range(num_layers)])
        self.decoder = nn.ModuleList([DecoderLayer(d_model, heads) for _ in range(num_layers)])
        self.logit = nn.Linear(d_model, self.vocab_size)
        
    def encode(self, src_words, src_mask):
        src_embeddings = self.embed(src_words)
        
        for layer in self.encoder:
            src_embeddings = layer(src_embeddings, src_mask)
            
        return src_embeddings

    def decode(self, target_words, target_mask, src_embeddings, src_mask):
        tgt_embeddings = self.embed(target_words)
        for layer in self.decoder:
            tgt_embeddings = layer(tgt_embeddings, src_embeddings, src_mask, target_mask)
        return tgt_embeddings

    def forward(self, src_words, src_mask, target_words, target_mask):
        encoded = self.encode(src_words, src_mask)
        decoded = self.decode(target_words, target_mask, encoded, src_mask)
        out = F.log_softmax(self.logit(decoded), dim = 2)
        return out
            

In [23]:
class AdamWarmup:
    
    def __init__(self, model_size, warmup_steps, optimizer):
        
        self.model_size = model_size
        self.warmup_steps = warmup_steps
        self.optimizer = optimizer
        self.current_step = 0
        self.lr = 0
    
    def get_lr(self):
        return self.model_size ** (-0.5) * min(self.current_step ** (-0.5), self.current_step * self.warmup_steps ** (-1.5))
    
    def step(self):
        
        self.current_step += 1
        lr = self.get_lr()
        for param_group in self.optimizer.param_groups:
            param_group['lr'] = lr
            
        self.lr = lr
        self.optimizer.step()

In [24]:
class LossWithLS(nn.Module):
    
    def __init__(self, size, smooth):
        super(LossWithLS, self).__init__()
        self.criterion = nn.KLDivLoss(size_average=False, reduce=False)
        self.confidence = 1.0 - smooth
        self.smooth = smooth
        self.size = size

    def forward(self, prediction, target, mask):

        prediction = prediction.view(-1, prediction.size(-1))   # (batch_size * max_words, vocab_size)
        target = target.contiguous().view(-1)   # (batch_size * max_words)
        mask = mask.float()
        mask = mask.view(-1)       # (batch_size * max_words)
        labels = prediction.data.clone()
        labels.fill_(self.smooth / (self.size - 1))
        labels.scatter_(1, target.data.unsqueeze(1), self.confidence)
        loss = self.criterion(prediction, labels)    # (batch_size * max_words, vocab_size)
        loss = (loss.sum(1) * mask).sum() / mask.sum()
        return loss
    

# Training

In [25]:
128 / 8

16.0

In [26]:
d_model = 128
heads = 4
num_layers = 2
num_layers = 2
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
epochs = 10
vocab_len = len(tokenizer.get_vocab())

transformer = Transformer(d_model = d_model , heads = heads, num_layers = num_layers, vocab_size = vocab_len)
transformer = transformer.to(device)
adam_optimizer = torch.optim.Adam(transformer.parameters())
transformer_optimizer = AdamWarmup(model_size = d_model, warmup_steps = 4000, optimizer = adam_optimizer)
criterion = LossWithLS(vocab_len, 0.1)

d:\anaconda\envs\Pray\Lib\site-packages\torch\nn\_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))


In [27]:
next(iter(train_loader.dataset))[1].shape

torch.Size([128])

In [28]:
def train(train_loader, transformer, criterion, epoch):
    
    transformer.train()
    sum_loss = 0
    count = 0
    
    for i, (inputs, output) in enumerate(train_loader):
        
        samples = inputs.shape[0]
        
        inputs = inputs.to(device)
        output = output.to(device)
        
        output_in = output
        output_target = output
        
        
        inputs_mask, output_in_mask, output_target_mask = create_masks(inputs, output_in, output_target)
        
        out = transformer(inputs, inputs_mask, output_in, output_in_mask)
        
        loss = criterion(out, output_target, output_target_mask)
        transformer_optimizer.optimizer.zero_grad()
        loss.backward()
        transformer_optimizer.step()
        
        sum_loss += loss.item() * samples
        count += samples
        
        
        
        if i % 20 == 0:
            print("Epoch [{}][{}/{}]\tLoss: {:.3f}".format(epoch, i, len(train_loader), sum_loss/count))

In [29]:
torch.Tensor((64, 128, 128)) * math.sqrt(128)

tensor([ 724.0773, 1448.1547, 1448.1547])

In [30]:
for epoch in range(epochs):
    
    train(train_loader, transformer, criterion, epoch)
    
    state = {'epoch': epoch, 'transformer': transformer, 'transformer_optimizer': transformer_optimizer}
    torch.save(state, 'checkpoint_' + str(epoch) + '.pth.tar')

Epoch [0][0/453]	Loss: 9.633
Epoch [0][20/453]	Loss: 9.631
Epoch [0][40/453]	Loss: 9.548
Epoch [0][60/453]	Loss: 9.424
Epoch [0][80/453]	Loss: 9.279
Epoch [0][100/453]	Loss: 9.126
Epoch [0][120/453]	Loss: 8.961
Epoch [0][140/453]	Loss: 8.783
Epoch [0][160/453]	Loss: 8.595
Epoch [0][180/453]	Loss: 8.407
Epoch [0][200/453]	Loss: 8.217
Epoch [0][220/453]	Loss: 8.027
Epoch [0][240/453]	Loss: 7.838
Epoch [0][260/453]	Loss: 7.647
Epoch [0][280/453]	Loss: 7.458
Epoch [0][300/453]	Loss: 7.268
Epoch [0][320/453]	Loss: 7.078
Epoch [0][340/453]	Loss: 6.889
Epoch [0][360/453]	Loss: 6.699
Epoch [0][380/453]	Loss: 6.513
Epoch [0][400/453]	Loss: 6.327
Epoch [0][420/453]	Loss: 6.146
Epoch [0][440/453]	Loss: 5.971
Epoch [1][0/453]	Loss: 2.064
Epoch [1][20/453]	Loss: 1.935
Epoch [1][40/453]	Loss: 1.844
Epoch [1][60/453]	Loss: 1.772
Epoch [1][80/453]	Loss: 1.701
Epoch [1][100/453]	Loss: 1.635
Epoch [1][120/453]	Loss: 1.574
Epoch [1][140/453]	Loss: 1.513
Epoch [1][160/453]	Loss: 1.460
Epoch [1][180/453]	L

KeyboardInterrupt: 

# Evaluate

In [26]:
d_model = 128
heads = 4
num_layers = 2
num_layers = 2
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
epochs = 10
vocab_len = len(tokenizer.get_vocab()) + 1


transformer = Transformer(d_model = d_model , heads = heads, num_layers = num_layers, vocab_size = vocab_len)
transformer = transformer.to(device)
# adam_optimizer = torch.optim.Adam(transformer.parameters())
# transformer_optimizer = AdamWarmup(model_size = d_model, warmup_steps = 4000, optimizer = adam_optimizer)
criterion = LossWithLS(vocab_len, 0.1)

d:\anaconda\envs\Pray\Lib\site-packages\torch\nn\_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))


In [27]:
import torch
from copy import deepcopy

# 저장된 파일을 불러올 때
checkpoint = torch.load('checkpoint_7.pth.tar')

# 불러온 checkpoint에서 모델 상태나 다른 필요한 요소들을 추출할 수 있습니다.
transformer = deepcopy(checkpoint['transformer'])

# 모델을 evaluation 모드로 설정 (필요에 따라)


In [28]:
device

device(type='cuda')

In [37]:
dict = tokenizer.get_vocab()

In [38]:
len(test_loader.dataset)

1000

In [39]:
bb = {v:k for k,v in tokenizer.get_vocab().items()} 
start_token = dict['</s>']

In [40]:
dict['<start>'] = 58101

In [41]:
start_token = dict['<start>']

In [59]:
def evaluate(transformer, question, question_mask, max_len, dict):

    transformer.eval()

    bb = {v:k for k,v in tokenizer.get_vocab().items()} 
    start_token = 58101
    encoded = transformer.encode(question, question_mask)
    words = torch.LongTensor([[start_token]]).to(device)

    for step in range(max_len - 1):
        size = words.shape[1]
        target_mask = torch.triu(torch.ones(size, size)).transpose(0, 1).type(dtype=torch.uint8)
        target_mask = target_mask.to(device).unsqueeze(0).unsqueeze(0)
        decoded = transformer.decode(words, target_mask, encoded, question_mask)
        predictions = transformer.logit(decoded[:, -1])
        _, next_word = torch.max(predictions[:,:-1], dim = 1)
        
        next_word = next_word.item()
  
        words = torch.cat([words, torch.LongTensor([[next_word]]).to(device)], dim = 1)   # (1,step+2)


    if words.dim() == 2:
        words = words.squeeze(0)
        words = words.tolist()

    print(words)

    return sentence

In [61]:
bb[18]

'e'

In [65]:
next(iter(train_loader))[0][0]

tensor([58101,    93,  2896,   564, 15202,  1729,    14, 24185,  4470,     2,
        16209,    79,    14, 27192,    18,     7, 24185,  2729,    56,     3,
            0, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100,
        58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100,
        58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100,
        58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100,
        58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100,
        58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100,
        58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100,
        58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100,
        58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100,
        58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100,
        58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100])

In [66]:
max_len = 128
enc_qus = [58101,    93,  2896,   564, 15202,  1729,    14, 24185,  4470,     2,
        16209,    79,    14, 27192,    18,     7, 24185,  2729,    56,     3,
            0, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100,
        58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100,
        58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100,
        58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100,
        58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100,
        58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100,
        58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100,
        58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100,
        58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100,
        58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100,
        58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100]
question = torch.LongTensor(enc_qus).to(device).unsqueeze(0)
question_mask = (question!=58100).to(device).unsqueeze(1).unsqueeze(1)
sentence = evaluate(transformer, question, question_mask, int(max_len), dict)
print(sentence)

[58101, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18]


NameError: name 'sentence' is not defined

In [104]:
question_mask

tensor([[[[ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
            True,  True,  True,  True,  True,  True,  True, False, False, False,
           False, False, False, False, False, False, False, False, False, False,
           False, False, False, False, False, False, False, False, False, False,
           False, False, False, False, False, False, False, False, False, False,
           False, False, False, False, False, False, False, False, False, False,
           False, False, False, False, False, False, False, False, False, False,
           False, False, False, False, False, False, False, False, False, False,
           False, False, False, False, False, False, False, False, False, False,
           False, False, False, False, False, False, False, False, False, False,
           False, False, False, False, False, False, False, False, False, False,
           False, False, False, False, False, False, False, False, False, False,
           False, False, Fal

In [ ]:
sentence

''

# 끝

In [ ]:
feg = []
for ww in A.detach().clone().cpu().numpy():
    feg_small = []
    for www in ww:
        if www == 0:
            break
        else:
            feg_small.append(bb[www])
        
    feg.append(feg_small)

In [ ]:
feg = []
for ww in A.detach().clone().cpu().numpy():
    feg_small = []
    for www in ww:
        if www == 0:
            break
        else:
            feg_small.append(bb[www])
        
    feg.append(feg_small)

In [ ]:
torch.argmax(torch.exp(out.detach().clone()),dim = -1).shape

torch.Size([64, 128])

In [39]:
fff = [tokenizer.tokenize(i) for i in dataset3['test']['de']]

In [53]:
i == j

True

In [49]:
from nltk.translate.bleu_score import corpus_bleu

scores = []
for i,j in tqdm(zip(feg, fff)):
    score1 = corpus_bleu(i, j)
    scores.append(score1)


0it [00:00, ?it/s]d:\anaconda\envs\Pray\Lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
d:\anaconda\envs\Pray\Lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
d:\anaconda\envs\Pray\Lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()